In [1]:
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

In [22]:
df = pd.DataFrame(pd.read_csv('merged.csv'))
df = df[df['Population'] > 0]
df['Count'] = df['Count'].fillna(0)
df.head()

,Race,Age,Gender,Location,Year,Count,Population
0,Asian,15–19,F,F4,2010,5.0,134.0
1,Asian,15–19,F,F4,2011,5.0,120.0
2,Asian,15–19,F,F4,2012,2.0,117.0
3,Asian,15–19,F,F4,2013,3.0,108.0
4,Asian,15–19,F,F4,2014,3.0,118.0


## First, 

In [115]:
poisson_res = smf.glm(formula='Count ~ C(Race) + C(Age) + C(Gender) + C(Location) + C(Year) -1', data=df, family=sm.families.Poisson(), 
              exposure=df.Population).fit()

In [116]:
df['mu'] = poisson_res.mu

In [117]:
# https://dius.com.au/2017/08/03/using-statsmodels-glms-to-model-beverage-consumption/

def response(row):
    "Calculate response observation for Cameron-Trivedi dispersion test"
    y = row['Count']
    m = row['mu']
    return ((y - m)**2 - y) / m

response_var = df.apply(response, axis=1)
df['response_var'] = response_var

ols_res = smf.ols('response_var ~ mu - 1', df).fit()
alpha = ols_res.params[0]

print('Overdispersion phi = ', 1.0/alpha)

Overdispersion phi =  13.481795676247309


In [118]:
res = smf.glm(formula='Count ~ C(Race) + C(Age) + C(Gender) + C(Location) + C(Year) -1', data=df, family=sm.families.NegativeBinomial(alpha=alpha), 
              exposure=df.Population).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Count   No. Observations:                13200
Model:                            GLM   Df Residuals:                    13132
Model Family:        NegativeBinomial   Df Model:                           67
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -73478.
Date:                Wed, 10 Mar 2021   Deviance:                       36140.
Time:                        17:23:37   Pearson chi2:                 3.51e+04
No. Iterations:                    12                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(Race)[Asian]       -2.3671      0.024    -97.578      0.000      -2.415      -2.320
C(Race)[Black]       -1.5080      0.023    -64.911      0.000      -1.553      -1.462
C(Race)[Hispanic]    -2.0636      0.023    -88.330      0.000      -2.109      -2.018
C(Race)[White]       -1.9028      0.023    -82.247      0.000      -1.948      -1.857
C(Age)[T.20–29]       0.8646      0.009     98.357      0.000       0.847       0.882
C(Age)[T.30–39]       0.6507      0.009     73.693      0.000       0.633       0.668
C(Age)[T.40–49]       0.4085      0.009     45.760      0.000       0.391       0.426
C(Age)[T.50+]        -0.3994      0.009    -43.935      0.000      -0.417      -0.382
C(Gender)[T.M]        0.7844      0.005    143.209      0.000       0.774       0.795
C(Location)[T.A2]    -0.3767      0.030    -12.425      0.000      -0.436      -0.317
C(Location)[T.A3]    -0.5577      0.029    -18.929      0.000      -0.615      -0.500
C(Location)[T.A4]     0.0104      0.029      0.357      0.721      -0.047       0.067
C(Location)[T.A5]    -1.1572      0.029    -40.266      0.000      -1.214      -1.101
C(Location)[T.A6]    -1.2165      0.029    -41.500      0.000      -1.274      -1.159
C(Location)[T.A7]    -0.5481      0.030    -18.482      0.000      -0.606      -0.490
C(Location)[T.A8]    -0.5028      0.030    -16.799      0.000      -0.562      -0.444
C(Location)[T.B1]    -2.9832      0.030    -99.941      0.000      -3.042      -2.925
C(Location)[T.B2]    -0.5172      0.029    -17.616      0.000      -0.575      -0.460
C(Location)[T.B3]    -1.8201      0.029    -61.915      0.000      -1.878      -1.762
C(Location)[T.B4]    -0.5557      0.029    -19.160      0.000      -0.613      -0.499
C(Location)[T.B5]    -0.1416      0.029     -4.842      0.000      -0.199      -0.084
C(Location)[T.B6]    -1.5319      0.029    -53.046      0.000      -1.588      -1.475
C(Location)[T.B7]    -0.7341      0.029    -25.260      0.000      -0.791      -0.677
C(Location)[T.B8]    -0.4894      0.029    -17.091      0.000      -0.546      -0.433
C(Location)[T.C1]    -0.8852      0.029    -30.323      0.000      -0.942      -0.828
C(Location)[T.C2]    -1.4214      0.029    -48.372      0.000      -1.479      -1.364
C(Location)[T.C3]    -2.2789      0.028    -81.679      0.000      -2.334      -2.224
C(Location)[T.C4]    -0.7849      0.029    -26.773      0.000      -0.842      -0.727
C(Location)[T.C5]    -1.1189      0.029    -37.990      0.000      -1.177      -1.061
C(Location)[T.C6]    -0.8994      0.029    -31.170      0.000      -0.956      -0.843
C(Location)[T.C7]    -2.1457      0.029    -75.277      0.000      -2.202      -2.090
C(Location)[T.C8]    -0.3221      0.030    -10.780      0.000      -0.381      -0.264
C(Location)[T.D1]    -0.5341      0.029    -18.298      0.000      -0.591

In [119]:
for race in ['Asian', 'White', 'Black', 'Hispanic']:
    print('{} Coef = '.format(race) , res.params['C(Race)[{}]'.format(race)] - res.params['C(Race)[White]'])

Asian Coef =  -0.4643313097454096
White Coef =  0.0
Black Coef =  0.3948316581197282
Hispanic Coef =  -0.16076662747919035


In [120]:
for gender in ['M']:
    print('{} Coef = '.format(gender) , res.params['C(Gender)[T.{}]'.format(gender)])

M Coef =  0.7844337750162934


In [121]:
for age in ['20–29', '30–39', '40–49', '50+']:
    print('{} Coef = '.format(age) ,  res.params['C(Age)[T.{}]'.format(age)])

20–29 Coef =  0.864643384333091
30–39 Coef =  0.6506809553323125
40–49 Coef =  0.40847987929140556
50+ Coef =  -0.39943695173083493


In [106]:
res_df = pd.DataFrame({'Category': ['Asian', 'Black', 'Hispanic', 'M', '20-29', '30-39', '40-49', '50+'], 
                       'Coef': [-0.464, 0.395, -0.160, 0.784 ,0.865, 0.651, 0.408, -0.40],
                       'Stderr': [.024, .023, .023, .005, .009, .009, .009, .009]})

In [108]:
res_df['Rate'] = res_df['Coef'].apply(np.exp)

In [109]:
res_df

,Category,Coef,Stderr,Rate
0,Asian,-0.464,0.024,0.628764
1,Black,0.395,0.023,1.484384
2,Hispanic,-0.160,0.023,0.852144
3,M,0.784,0.005,2.190216
4,20-29,0.865,0.009,2.375006
5,30-39,0.651,0.009,1.917457
6,40-49,0.408,0.009,1.503807
7,50+,-0.400,0.009,0.670320


## Q3C
C. Give three distinct potential reasons for the racial disparity in stop rate as measured in part B.

1. Racial bias in policing -- law enforcement officers may be more likely to stop drivers who are Black compared to other races.
2. Uneven distribution of policing -- officers may be more present in urban regions, for example, or other regions with particular racial distributions.
3. re: 'Sundown Town' -- if you are Black in a predominantly White town, you might be particularly likely to be stopped, and if you are Black in a predominantly Black town, you might still be likely to be stopped (for reason number 2). 

In [122]:
df = pd.DataFrame(pd.read_csv('merged.csv'))
df = df[df['Population'] > 0]
df['Count'] = df['Count'].fillna(0)

In [125]:
df.to_csv('q3.csv')